# EDA Project 1

## Stakeholder
**Erin Robinson** 
- Buyer 
- Invest in poor neighbourhood, buying & selling, costs back + little profit 
- socially responsible

***subtitle***

## Table of Contents
0. Set-up
1. Exploring the data set
2. Building hypotheses
3. Testing hypotheses & building a model
4. Visualizing results
5. Drawing conclusions

### 0. Set-up

***Preparing data frame(s) and making data accessible / Clean-up***

---

In [2]:
# import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px
import seaborn as sns
from datetime import datetime, date, time, timedelta

# plotting by states -- not yet functional !!!
import plotly.figure_factory as ff

In [3]:
# create main data frame
df = pd.read_csv('../Data/King_County_House_prices_dataset.csv')

As our stakeholder is interested to invest in **poor neighborhoods**, it appears reasonable to supplement our data set through domain knowledge:

In [4]:
# supplemetary data
# source: http://www.zipatlas.com/us/wa/zip-code-comparison/median-household-income.htm, viewed on 15.12.2021, 17:00
median_income_by_zip = pd.read_csv('../Data/state-geocodes-v2013.csv', sep=';')
median_income_by_zip.head()

,id,zipcode,Location,City,Population,median_income,National Rank
0,1,98039,47626035 -122233707,Medina Washington,3050,132665,#46
1,2,98053,47655058 -122025817,Redmond Washington,31203,96028,#335
2,3,98364,47855439 -122595728,Port Gamble Washington,102,94366,#364
3,4,98040,47562356 -122226514,Mercer Island Washington,22036,91904,#415
4,5,98029,47578444 -122013924,Issaquah Washington,28427,91146,#430


In [5]:
# add supplementary data to dataframe
df = pd.merge(df, median_income_by_zip[['zipcode', 'median_income']], on='zipcode')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,median_income
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,47461
1,4060000240,6/23/2014,205425.0,2,1.00,880,6780,1.0,0.0,0.0,...,880,0.0,1945,0.0,98178,47.5009,-122.248,1190,6780,47461
2,4058801670,7/17/2014,445000.0,3,2.25,2100,8201,1.0,NaN,2.0,...,1620,480.0,1967,0.0,98178,47.5091,-122.244,2660,8712,47461
3,2976800796,9/25/2014,236000.0,3,1.00,1300,5898,1.0,NaN,0.0,...,1300,0.0,1961,NaN,98178,47.5053,-122.255,1320,7619,47461
4,6874200960,2/27/2015,170000.0,2,1.00,860,5265,1.0,0.0,0.0,...,860,0.0,1931,0.0,98178,47.5048,-122.272,1650,8775,47461


In [6]:
# Prepare relative values of grade and condition categories
grade_rel = df[['id', 'grade']].groupby('grade').count().reset_index()
grade_rel['grade_rel'] = grade_rel.id / grade_rel.id.sum()

cond_rel = df[['id', 'condition']].groupby('condition').count().reset_index()
cond_rel['cond_rel'] = cond_rel.id / cond_rel.id.sum()


In [7]:
# Assign pricing brakets to the data set
# 1 = < 100,000
# 2 = 100,000 - 150,000
# 3 = 150,000 - 200,000
# 4 = 200,000 - 250,000
# 5 = 250,000 - 300,000
# 6 = 300,000 - 350,000
# 7 = 350,000 - 400,000
# 8 = 400,000 - 450,000
# 9 = 450,000 - 500,000
# 10 = > 500,000 

brackets_price = {100000: '0 - 100,000',       150000: '100,000 - 150,000',
                  200000: '150,000 - 200,000', 250000: '200,000 - 250,000',
                  300000: '250,000 - 300,000', 350000: '300,000 - 350,000',
                  400000: '350,000 - 400,000', 450000: '400,000 - 450,000',
                  500000: '450,000 - 500,000', df.price.max() + 1: '> 500,000'}

def find_category(price, brackets):
    for b in brackets.keys():
        if price < b:
            return brackets[b]
        
# add price category column to data frame
df['price_category'] = df.price.apply(lambda x: find_category(x, brackets_price))

In [14]:
# Assign income brakets to the data set
# 1 = < 35,000
# 2 = 35,000 - 40,000
# 3 = 40,000 - 45,000
# 4 = 45,000 - 50,000
# 5 = 50,000 - 55,000
# 6 = 55,000 - 60,000
# 7 = 60,000 - 65,000
# 8 = 65,000 - 70,000
# 9 = 70,000 - 75,000
# 10 = > 75,000 

brackets_inc = {35000: '0 - 35,000',      40000: '35,000 - 40,000',
                45000: '40,000 - 45,000', 50000: '45,000 - 50,000',
                55000: '50,000 - 55,000', 60000: '55,000 - 60,000',
                65000: '60,000 - 65,000', 70000: '65,000 - 70,000',
                75000: '70,000 - 75,000', df.median_income.max() + 1: '> 75,000'}

def find_category(price, brackets):
    for b in brackets.keys():
        if price < b:
            return brackets[b]
        
# add price category column to data frame
df['income_category'] = df.median_income.apply(lambda x: find_category(x, brackets_inc))

In [16]:
# dissect 'date' column
# convert date string to date class
df['date_'] = df.date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())
# extract from date year and month
df['date_year'] = df.date_.apply(lambda x: x.year)
df['date_month'] = df.date_.apply(lambda x: x.month)

#df['date_ym'] = df.date_.dt.to_period('M')
#df.date.dt.to_period(‘M’)
#df.drop(['date_year_month'], axis=1, inplace=True)


In [17]:
# add additional information with respect to year build and renovation
df['yr_renovated'] = df['yr_renovated'].fillna(0)
# set boolean if renovated
df['renovated'] = df.yr_renovated.apply(lambda x: 0 if x == 0 else 1)
df['built_renov_diff'] = abs(df.yr_built - df.yr_renovated)
#df['built_renov_diff'] = df[['yr_built', 'yr_renovated']].apply(lambda x,y: 0 if y == 0 else y-x)#abs(df.yr_built - df.yr_renovated)
df[['yr_built', 'yr_renovated', 'renovated', 'built_renov_diff']].head(15)

,yr_built,yr_renovated,renovated,built_renov_diff
0,1955,0.0,0,1955.0
1,1945,0.0,0,1945.0
2,1967,0.0,0,1967.0
3,1961,0.0,0,1961.0
4,1931,0.0,0,1931.0
5,1954,0.0,0,1954.0
6,1943,0.0,0,1943.0
7,1943,0.0,0,1943.0
8,1920,0.0,0,1920.0
9,1930,1983.0,1,53.0


As the data frame is now prepared and cleaned up, we can move to the next step: exploring the data to extract first information and build our hypotheses.

In [18]:
# write complete data frame into new csv file
df.to_csv('../data/real-estate-KC.csv', mode='w+')

<pre></pre>

***Variables explained***

---
* **id** - unique identified for a house
* **dateDate** - house was sold
* **pricePrice** -  is prediction target
* **bedroomsNumber** -  # of bedrooms
* **bathroomsNumber** - # of bathrooms
* **sqft_livingsquare** -  footage of the home
* **sqft_lotsquare** -  footage of the lot
* **floorsTotal** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

## Additional install requirements

- plotly-geo
- geopandas==0.3.0
- pyshp==1.2.10
- shapely==1.6.3